In [17]:
import pandas as pd
import numpy as np

df = pd.read_csv('E:\\Downloads\\train2_irf_0000_corr98.csv', dtype='int8')
#y = pd.read_csv('E:\\Downloads\\y2.csv', sep=';', usecols=[1], dtype='int8').values.ravel()
y = pd.read_csv('E:\\Downloads\\y2.csv', sep=';', usecols=[1], dtype='int8')

print(df.shape, y.shape)

(26495, 8065) (26495, 1)


In [21]:
from keras.models import load_model
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import train_test_split
#best_model_14-0.778.h5 test_irf_0000_corr98
file = 'E:\\Downloads\\best_model_14-0.778.h5'
test = pd.read_csv('E:\\Downloads\\test_irf_0000_corr98.csv', dtype='int8')
model = load_model(file)
valid_preds = model.predict_proba(test, verbose=0)

#roc = roc_auc_score(y, valid_preds)
#print(roc)

In [22]:
submission = pd.DataFrame(columns={'Ids', 'TARGET'})
submission['Ids'] = [i for i in range(26500,39750)]
submission['TARGET'] = valid_preds
submission.to_csv("E:\\Downloads\\submission.csv", sep=';', index=False, header={'Ids', 'TARGET'})

In [26]:
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import cross_val_score
from keras.models import Sequential
from keras.layers import Dense, Dropout, GaussianNoise
from keras.callbacks import EarlyStopping, ModelCheckpoint
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold

X_train, X_test, y_train, y_test = train_test_split(df, y, test_size=.3, stratify=y)

def build_classifier():
    classifier = Sequential()
    classifier.add(Dense(units = 1024, kernel_initializer = 'lecun_uniform', activation = 'relu', input_dim = X_train.shape[1]))
    classifier.add(Dropout(0.1))
#    classifier.add(Dense(units = 1024, kernel_initializer = 'lecun_uniform', activation = 'relu'))
#    classifier.add(Dense(units = 512, kernel_initializer = 'lecun_uniform', activation = 'relu'))
    classifier.add(Dense(units = 256, kernel_initializer = 'lecun_uniform', activation = 'relu'))
    classifier.add(GaussianNoise(stddev=0.01))
    classifier.add(Dense(units = 128, kernel_initializer = 'lecun_uniform', activation = 'relu'))
    classifier.add(Dense(units = 64, kernel_initializer = 'lecun_uniform', activation = 'relu'))
    classifier.add(Dense(units = 1, kernel_initializer = 'lecun_uniform', activation = 'sigmoid'))
    classifier.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])
    return classifier

av_roc = 0.
callbacks = [EarlyStopping(monitor='val_acc', patience=6, verbose=1), ModelCheckpoint(filepath='E:\\Downloads\\best_model_{epoch:02d}-{val_acc:.3f}.h5', monitor='val_acc', save_best_only=True)]

#classifier = KerasClassifier(build_fn = build_classifier, epochs=100, batch_size=300)
classifier = build_classifier()
classifier.fit(X_train, y_train, validation_data=(X_test, y_test), shuffle=True, epochs=80, batch_size=750, callbacks=callbacks)

#valid_preds = classifier.predict_proba(X_test, verbose=0)[:, 1]
valid_preds = classifier.predict_proba(X_test, verbose=0)

roc = roc_auc_score(y_test, valid_preds)
print(roc)

#valid_preds = classifier.predict_proba(X_test, verbose=0)
#valid_preds = valid_preds[:, 1]
#roc = roc_auc_score(y_test, valid_preds)
#print("ROC:", roc)
#av_roc += roc

#print('Average ROC:', av_roc/nb_folds)

Train on 18546 samples, validate on 7949 samples
Epoch 1/80
18546/18546 [==============================] - 3s 186us/step - loss: 0.7590 - acc: 0.5245 - val_loss: 0.6672 - val_acc: 0.5948
Epoch 2/80
18546/18546 [==============================] - 2s 121us/step - loss: 0.6241 - acc: 0.6529 - val_loss: 0.5875 - val_acc: 0.6948
Epoch 3/80
18546/18546 [==============================] - 2s 122us/step - loss: 0.5509 - acc: 0.7170 - val_loss: 0.5388 - val_acc: 0.7219
Epoch 4/80
18546/18546 [==============================] - 2s 122us/step - loss: 0.5064 - acc: 0.7525 - val_loss: 0.5079 - val_acc: 0.7479
Epoch 5/80
18546/18546 [==============================] - 2s 123us/step - loss: 0.4763 - acc: 0.7747 - val_loss: 0.5131 - val_acc: 0.7455
Epoch 6/80
18546/18546 [==============================] - 2s 121us/step - loss: 0.4446 - acc: 0.7941 - val_loss: 0.4901 - val_acc: 0.7587
Epoch 7/80
18546/18546 [==============================] - 2s 122us/step - loss: 0.4482 - acc: 0.7898 - val_loss: 0.4841 - v

In [8]:
    preds = model.predict_proba(X_test, verbose=0)[:, 1]
    submission = pd.DataFrame(preds, index=ids, columns=['target'])
    submission.to_csv('Keras_BTB.csv')    

NameError: name 'model' is not defined

In [ ]:
kfold = StratifiedKFold(n_splits=3, shuffle=True, random_state=42)
aucs = []
tprs = []
mean_fpr = np.linspace(0, 1, 100)

for train_index, test_index in kfold.split(df, y):
        classifier.fit(df.iloc[train_index], y.iloc[train_index], validation_data=(df.iloc[test_index], y.iloc[test_index]), epochs=100, batch_size=180, callbacks=callbacks)
    valid_preds = classifier.predict_proba(df.iloc[test_index], verbose=0)[:, 1]
    
    results = cross_val_score(classifier, df, y, cv=kfold, scoring = 'roc_auc', fit_params={'callbacks': callbacks})
    fpr, tpr, thresholds = roc_curve(y.iloc[test_index], probas_)
    tprs.append(interp(mean_fpr, fpr, tpr))
    tprs[-1][0] = 0.0
    roc_auc = auc(fpr, tpr)
    aucs.append(roc_auc)
    
mean_tpr = np.mean(tprs, axis=0)
mean_tpr[-1] = 1.0
mean_auc = auc(mean_fpr, mean_tpr)
